# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile
from collections import Counter

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('C:/Users/4stra/Documents/pyton/FDAN20/env/edan20/notebooks/Selma/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [6]:
words = tokenize(corpus)
words[:10]


['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
l=set(words)
k=len(list(l))
print(k)

44256


Lowercased text

In [8]:
words=tokenize(corpus.lower())
l=set(words)
k=len(list(l))
print(k)
# Write your code here

41032


### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
nonletter='[^\p{L}.:;!?]+'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
def clean (text):
    return re.sub(nonletter,r' ',text)
    

In [11]:
test_para ='En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden. Mosa underade wuuut!!! \
Sen frågade Fanny vad menar du ??? '





In [12]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. Mosa underade wuuut!!! Sen frågade Fanny vad menar du ??? '

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [13]:
sentence_boundaries='[.!?:;]+\s+(\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [14]:
sentence_markup=r' </s>\n<s> \1'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [15]:
text= re.sub(sentence_boundaries,sentence_markup,test_para)

In [16]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s> Mosa underade wuuut </s>
<s> Sen frågade Fanny vad menar du ??? 


Insert markup codes in the beginning and end of the text

In [17]:
text= "<s> "+text+" </s>" 

In [18]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s> Mosa underade wuuut </s>
<s> Sen frågade Fanny vad menar du ???  </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [19]:
text=re.sub(r'[.!?:;]','',text)

text=re.sub(r'\s{2,}',' ',text)

In [20]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s> Mosa underade wuuut </s>
<s> Sen frågade Fanny vad menar du </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [21]:
def segment_sentences(text):
 
    text= re.sub(sentence_boundaries,sentence_markup,text)
    text=text.lower()
    text= "<s> "+text+" </s>"
    text=re.sub(r'[.?!:;]','',text)
    text=re.sub(r'\s{2,}',' ',text)
    return text 

In [22]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s> mosa underade wuuut </s>
<s> sen frågade fanny vad menar du </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [23]:
corpus=segment_sentences(clean(corpus))

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [24]:
print(corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [25]:
print(len(words))

words=re.findall(r'(\p{L}+|<s>|</s>)',corpus)
print(len(words))

923485
1041560


The five last lines of the corpus should like this:

In [26]:
print(words[-101:])


['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [27]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [28]:
frequency = unigrams(words)
list(frequency.items())#[:20]


[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2),
 ('signade', 3),
 ('dag', 942),
 ('som', 16288),
 ('vi', 2105),
 ('nu', 4084),
 ('här', 2107),
 ('se', 1989),
 ('av', 5435),
 ('himmelen', 80),
 ('till', 9139),
 ('oss', 974),
 ('nedkomma', 2),
 ('han', 21589),
 ('blive', 1),
 ('säll', 7),
 ('låte', 2),
 ('sig', 9250),
 ('te', 220),
 ('alla', 2355),
 ('glädje', 294),
 ('och', 36356),
 ('fromma', 29),
 ('ja', 938),
 ('herren', 23),
 ('högste', 7),
 ('i', 16508),
 ('för', 9443),
 ('synder', 6),
 ('sorger', 15),
 ('bevare', 18),
 ('men', 8144),
 ('såsom', 271),
 ('en', 13514),
 ('fågel', 49),
 ('mot', 1526),
 ('himmelens', 28),
 ('höjd', 23),
 ('lyfter', 29),
 ('på', 14250),
 ('lediga',

#### Bigrams

In [29]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [31]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]


[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [32]:

def unigram_lm(frequencey, send_words):
    #detta blir lite fel pga 
    sum=0
    for f in frequencey.keys():
        sum= sum+frequencey[f]
    sum2=1

    lw=sum;
    print('=====================================================')
    print('wi    C(wi)    #words    P(wi)')
    print('=====================================================')
    for s in send_words:
        c=frequencey[s]
        p=c/lw
        sum2=sum2*p

        print(s,'    ',c,'    ',lw,'   ',p)
    
    entropy=-math.log(sum2,2)/len(send_words)
    perp=2**(entropy)
    print('=====================================================')
    
    print('Prob. unigrams:    ',sum2)
    print('Geometric mean prob.:',sum2**(1/len(send_words)))
    print('Entropy rate:     ',entropy )
    print('Perplexity:  ',perp)
    return(perp)
    

 
    

In [33]:
sentence = 'detta hände i början av adertonhundratjugutalet i en avlägsen socken i västra värmland </s>'
sent_words = sentence.split()
sent_words

['detta',
 'hände',
 'i',
 'början',
 'av',
 'adertonhundratjugutalet',
 'i',
 'en',
 'avlägsen',
 'socken',
 'i',
 'västra',
 'värmland',
 '</s>']

In [34]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi    C(wi)    #words    P(wi)
detta      2482      1041560     0.002382964015515189
hände      122      1041560     0.00011713199431621799
i      16508      1041560     0.015849302968623986
början      205      1041560     0.0001968201543838089
av      5435      1041560     0.005218134336956105
adertonhundratjugutalet      4      1041560     3.840393256269442e-06
i      16508      1041560     0.015849302968623986
en      13514      1041560     0.01297476861630631
avlägsen      5      1041560     4.800491570336803e-06
socken      65      1041560     6.240639041437844e-05
i      16508      1041560     0.015849302968623986
västra      50      1041560     4.800491570336803e-05
värmland      84      1041560     8.064825838165828e-05
</s>      59047      1041560     0.056690925150735434
Prob. unigrams:     3.7393205093489276e-45
Geometric mean prob.: 0.0006708540552614568
Entropy rate:      10.541713438238645
Perplexity:   1490.6371842833437


In [35]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

1490

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [36]:

def bigram_lm(frequency, frequency_bigrams, send_words):
    #detta blir lite fel pga 

    send_words.insert(0,'<s>')
    sum=0
    for f in frequency.keys():
        sum= sum+frequency[f]
        
    sum2=1

    lw=sum
    print('=====================================================')
    print('wi    wi+1    Ci,i+1    C(i)    P(wi+1|wi)')
    print('=====================================================')
    for s in range(0,len(send_words)-1):
        fw=send_words[s]
        sw=send_words[s+1]
        if (fw,sw) in frequency_bigrams.keys():
            ciip1=frequency_bigrams[(fw,sw)]
            ci=frequency[fw]
            p=ciip1/ci
            sum2=sum2*p
            print(fw,'    ',sw,'    ',ciip1,'    ',ci,'   ',p)
    
        else:
            ciip1=0 
            ci=frequency[fw]
            p=frequency[sw]/lw
            sum2=sum2*p
            print(fw,'    ',sw,'    ',ciip1,'    ',ci,' 0.0   *backoff:',p)
            
    entropy=-math.log(sum2,2)/(len(send_words)-1)
    perp=2**(entropy)
    print('=====================================================')
    
    print('Prob. bigrams:    ',sum2)
    print('Geometric mean prob.:',sum2**(1/(len(send_words)-1)))
    print('Entropy rate:     ',entropy )
    print('Perplexity:  ',perp)
    return(perp)

In [37]:
print(sent_words)
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

['detta', 'hände', 'i', 'början', 'av', 'adertonhundratjugutalet', 'i', 'en', 'avlägsen', 'socken', 'i', 'västra', 'värmland', '</s>']
wi    wi+1    Ci,i+1    C(i)    P(wi+1|wi)
<s>      detta      298      59047     0.00504682710383254
detta      hände      7      2482     0.0028203062046736503
hände      i      1      122     0.00819672131147541
i      början      75      16508     0.004543251756724012
början      av      27      205     0.13170731707317074
av      adertonhundratjugutalet      1      5435     0.0001839926402943882
adertonhundratjugutalet      i      1      4     0.25
i      en      575      16508     0.03483159680155076
en      avlägsen      1      13514     7.399733609590054e-05
avlägsen      socken      1      5     0.2
socken      i      7      65     0.1076923076923077
i      västra      10      16508     0.000605766900896535
västra      värmland      3      50     0.06
värmland      </s>      24      84     0.2857142857142857
Prob. bigrams:     1.851249355079148

In [73]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

81

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [98]:
def trigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1],words[i+2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [99]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [118]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [119]:
cand_nbr = 5

In [120]:
temp=re.findall(r'\bde\p{L}*',corpus)
temp=list(set(temp))
temp2=[]
for t in temp:
    if ('<s>',t) in frequency_bigrams.keys():
        temp2.append(t)
temp2.sort()
temp2.sort(key=lambda x: frequency_bigrams[('<s>',x)],reverse=True)
current_word_predictions_1=temp2[:5]



In [121]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [122]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [123]:
tokens=tokenize(current_text)

In [124]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [125]:
temp=words #re.findall(r'\bde\p{L}*',corpus)
temp=list(set(temp))
temp2=[]
for t in temp:
    if ('var','en',t) in frequency_trigrams.keys():
        temp2.append(t)
temp2.sort()
temp2.sort(key=lambda x: frequency_trigrams[('var','en',x)],reverse=True)
next_word_predictions=temp2[:5]



In [126]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [127]:
current_text = "Det var en g".lower()

In [128]:
temp=re.findall(r'\bg\p{L}*',corpus)
temp=list(set(temp))
#print(temp)
temp2=[]
for t in temp:
    if ('var','en',t) in frequency_trigrams.keys():
        temp2.append(t)
temp2.sort()
temp2.sort(key=lambda x: frequency_trigrams[('var','en',x)],reverse=True)
current_word_predictions_2=temp2[:5]

#frequency_trigrams[('var','en','glädje',)]


In [129]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [130]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [131]:
STIL_ID = ["Fa0188ni-s", "Mo0708ho-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [132]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "detta", "denna"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [55]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [56]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [57]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': 'ef5370e5c56f10168ae79f3f8b2e09b2435b9c9b70c41e6b4313157b7dbff43d227c359e6774c5c9163347f3e1a6a0a1cb6301e80a1f7b2dfc647c15577c1909',
 'submission_id': '08986bf2-c8d4-40cd-84c8-9e5f84594908'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to include the unigram and bigram tables for five sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 24, 2021.